
### Description
This part of the project focuses on identifying useful features for obtaining an adequate forecasting model  for TSD. Feature engineering and model training has been combined into a cyclic process to check the effects of feartures on model performance.

## Part 3: Feature Engineering

**Relevant Libraries**

In [ ]:
%load_ext autoreload
%autoreload 2
import pandas as pd
import plotly.express as px
from plotly.subplots import make_subplots

import os
import sys
sys.path.append(os.path.abspath('../'))
from statsmodels.tsa.seasonal import seasonal_decompose, DecomposeResult
from src.features.feature_utils import create_lag_features, plot_correlation_matrix, plot_seasonal_decompose
from src.features.feature_utils import plot_acf_and_pacf
import warnings
warnings.filterwarnings('ignore')

In [ ]:
sys.path.append(os.path.abspath('../'))
sys.path

In [ ]:
project_root = sys.path[-1]
project_root

### Load Data

In [ ]:
df = pd.read_pickle("../data/interim/uk_data_processed_postEDA.pkl")
df

---------------------------------

### Checking Trends and Seasonality

In [ ]:
result = seasonal_decompose(df['tsd'], model='additive', period=365*48)
df.set_index('date', inplace=True)
fig = plot_seasonal_decompose(result, dates=df.index, target_name='tsd')
fig.show()

In [ ]:
fig_acf, fig_pacf = plot_acf_and_pacf(df['tsd'], lags=336)

In [ ]:
fig_acf


In [ ]:
fig_pacf

### Generate Features

In [ ]:
df.set_index('date', inplace=True)
proc_fe_df = create_lag_features(df, save_data=True)


In [ ]:
proc_fe_df.columns

## Part 4: Modelling 

In [ ]:
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from xgboost import XGBRegressor
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
from sklearn.model_selection import TimeSeriesSplit, GridSearchCV
from src.models.model_utils import model_evaluator, model_feature_importance



In [ ]:
fe_df = pd.read_pickle("../data/processed/uk_data_fe_processed.pkl")
fe_df

In [ ]:
fe_df.columns

In [ ]:

fe_df = fe_df[['tsd', 'lag_1day', 'lag_1hour', 'lag_1week', 'lag_1year', 'lag_2year', 'rolling_mean_1day']]
fig = plot_correlation_matrix(fe_df)
fig


Correlation matrix suggests 5 features:<br>
1. lag48 - 1 day shift 
2. lag336 - 1 week shift
3. lag1yr - 1 year shift 
4. rolling_mean_48  
5. lag2yr - 2 year shift 

This reduces the number of features considered from 12 (initial for gradient boost model fold 5) which had added benefit of faster model training, and less redundacy. 


I drop:<br>
1. temporal features, period, days, weeks, months, etc they have been poor in earlier gradient boost and recent modeling in this notebook. Corr matrix also shows poor correlation. 
2. rolling_std_48: its highly co-linear with rolling_mean_48 but poorly corr with tsd
3. lag3yr: similar reason as 2 above
4. lag5yr: similar reason as 2 above

In [ ]:

px.line(fe_df, x=fe_df.index, y=['tsd', 
                                 'lag_1hour','lag_1day','lag_1week','lag_1year','lag_2year','rolling_mean_1day'])



In [ ]:


features = [
    'lag_1day', 'lag_1hour', 'lag_1week', 'lag_1year', 'lag_2year', 'rolling_mean_1day'
]
target = 'tsd'
fe_df = fe_df.sort_index()
X = fe_df[features].dropna()
y = fe_df[target].dropna().loc[X.index]         
 
# Define models
rf_model = RandomForestRegressor(n_estimators=100, random_state=42)
gb_model = GradientBoostingRegressor(n_estimators=100, random_state=42)
xgb_model = XGBRegressor(n_estimators=100,random_state=42)

# Stores for model outputs 
rf_results = []
gb_results = []
xgb_results = []

#----------------------------------------------------------------------
# Create time-series split cross validation
tscv = TimeSeriesSplit(n_splits=5, test_size =48*365*1, gap=48)


In [ ]:

# Loop through the folds...
for fold, (train_idx, test_idx) in enumerate(tscv.split(X), start=1):
    print(f"Running Fold {fold}...")
    print('Spliting data into training and testing data subsets...')
    X_train, X_test = X.iloc[train_idx], X.iloc[test_idx]
    y_train, y_test = y.iloc[train_idx], y.iloc[test_idx]
               
    # Run Models
    try:
        # Random Forest
        print('Running Random Forest model...')
        rf_model.fit(X_train, y_train)
        rf_pred = rf_model.predict(X_test)
        rf_results.append(model_evaluator(fold, y_test, rf_pred, rf_model, 'random_forest'))  
    except Exception as e:
        print(f"Random Forest model failed on fold {fold}: {e}")
        
    try:
        # Gradient Boosting
        print('Running Gradient Boost model...')
        gb_model.fit(X_train, y_train)
        gb_pred = gb_model.predict(X_test)
        gb_results.append(model_evaluator(fold, y_test, gb_pred, gb_model, 'gradient_boost'))   
    except Exception as e:
        print(f"Gradient Boostng model failed on fold {fold}: {e}")
        
    try:
        # XGBoost
        print('Running XGBoost model...')
        xgb_model.fit(X_train, y_train)
        xgb_pred = xgb_model.predict(X_test)
        xgb_results.append(model_evaluator(fold, y_test, xgb_pred, xgb_model, 'xgboost'))
    except Exception as e:
        print(f"XGBoost model failed on fold {fold}: {e}")
          
# Model training complete
print('Model training complete')



In [ ]:
rf_results

random forest model fold 5   

In [ ]:
gb_results

In [ ]:
xgb_results

In [ ]:
# Best Model
best_model_vars = rf_results[-1]   # best model - rf_model fold 1 
best_model_02_vars = xgb_results[-1]  # 2nd best model - xgb_model fold 5
print(f"Result of the best model:{best_model_vars}")
print(f"Result of the 2nd best model:{best_model_02_vars}")



In [ ]:
X

In [ ]:
# Feature Importance - Best Model...
print('Generating feature importance from best model')
_ , fig = model_feature_importance(X,best_model_vars)
fig.show()



In [ ]:
print('Generating feature importance from 2nd best model')
_ , fig = model_feature_importance(X,best_model_02_vars)
fig.show()

In [ ]:
def plot_actual_vs_model_pred(model_vars, X, y):
    """
    This function plots a comparison on the actual and predicted data for model verification.
     
    Args:
    model_vars:         a dictionary containing the variables of the best trained model
    X:                  the features that was used for training the best model
    y:                  the target that was used for training the best model
    Return:
    fig:                the plotly plot output
    
    """
    train_set = 0
    test_set = 1
    
    
    fold_ct = model_vars['fold'] - 1 # 0, 1, 2, 3, or 4 representing fold 1, 2, 3, 4 or 5 respectively 
    scope_test_idx = list(tscv.split(X))[fold_ct][test_set]
    X_scope_test = X.iloc[scope_test_idx]
    y_scope_test = y.iloc[scope_test_idx]
    
    model_pred = model_vars['model'].predict(X_scope_test)
    fig = px.line(title=f"Actual vs Predicted TSD from fold {model_vars['fold']}")
    fig.add_scatter(y=y_scope_test.values, mode='lines', name='Actual')
    fig.add_scatter(y=model_pred, mode='lines', name=f"{model_vars['model_name']} Predicted")
    fig.update_layout(yaxis_title='Transmission Systems Demand (MW)')
    
    return fig
    
    

In [ ]:
model_vars = best_model_vars
fig = plot_actual_vs_model_pred(model_vars, X, y)

In [ ]:
fig

In [ ]:
model_vars = best_model_02_vars
fig = plot_actual_vs_model_pred(model_vars, X, y)
fig.show()

### Save model and its performance metrics 

In [ ]:
import joblib
import pickle


joblib.dump(best_model_vars['model'], f"{project_root}/models/{best_model_vars['model_name']}_best_model.pkl")
joblib.dump(best_model_02_vars['model'], f"{project_root}/models/{best_model_02_vars['model_name']}_2nd_best_model.pkl")


In [ ]:
# alternate - save model and its metrics 
best_01_and_02_models = [best_model_vars,best_model_02_vars]

model_save_filepath = f"{project_root}/models/best_rf_and_xgb_models.pkl"
with open(model_save_filepath, 'wb') as file:
    pickle.dump(best_01_and_02_models, file)

In [ ]:
# To load the models
with open(model_save_filepath,'rb') as file:
    loaded_models = pickle.load(file)

In [ ]:
loaded_models

In [ ]:
ml_model_vars = loaded_models[1]

In [ ]:
ml_model_vars['model_name']
float(ml_model_vars['MAE'])

In [ ]:
pd.DataFrame({'model_name': ml_model_vars['model_name'],
            'MAE': [float(ml_model_vars['MAE'])],
            'RMSE': [float(ml_model_vars['RMSE'])],
            'MAPE': [float(ml_model_vars['MAPE'])]
})